In [21]:
import pandas as pd
# events_df = pd.read_csv("https://api.mockaroo.com/api/3f045400?count=1000&key=9aadb790")
events_df = pd.read_csv("events.csv")

In [22]:
import ast
data = list(map(lambda x: ast.literal_eval(x), events_df['tags']))
data_set = []
for row in data:
    data_set += row
data_set = set(data_set)
data = list(map(lambda x: ', '.join(x), data))

In [23]:
import random
def get_random_tag(set: data_set):
    return list(data_set)[random.randint(0,len(data_set))]

In [24]:
def exclude_chars():
    # Exclude unnecessary characters
    exclude_chars = '\{\}[]\'\"'
    data = [x.translate(str.maketrans('','',exclude_chars)) for x in data]
    query = query.translate(str.maketrans('','',exclude_chars))

In [31]:
import tensorflow as tf

# Function to get the top n recommendations for a given document index
def get_recommendations_tfidf(query, data, top_n=1, output_mode='tf_idf'):
    # Create the TextVectorization layer
    vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=None,  # Default value, no maximum limit on vocabulary size
        output_mode=output_mode,
        output_sequence_length=None,  # Default value, variable length sequences
        pad_to_max_tokens=False,  # Don't pad sequences
        standardize='lower_and_strip_punctuation',  # Disable standardization
        split='whitespace',  # Split text by whitespace
    )

    # Fit and transform the data
    new_data = data + [query]
    # print(new_data)
    vectorizer.adapt(new_data)
    X = vectorizer(new_data)

    # Compute the pairwise cosine similarity
    cosine_sim = tf.linalg.matmul(X, X, transpose_b=True)

    # Get the pairwise cosine similarity scores for the document
    sim_scores = list(enumerate(cosine_sim[-1, :-1]))

    # Sort the documents by similarity score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    top_docs = sim_scores[:top_n]
    print(top_docs)

    # Return the indices and recommendations
    return [[new_data[i], i, float(j)] for i, j in top_docs]

def loop_print(arr,contains):
  for el in arr:
    print(el)
  print(len(set(i[0] for i in arr) & contains))
  print()

def check_contains(raw_string):
    words = raw_string.strip("{},").split()
    output = {datum for datum in data if all(word in datum for word in words)}
    
    for i in output:
        print(i)
    print()
    
    return output


In [33]:
query = ', '.join([get_random_tag(data_set) for i in range(3)])
query = "Building"
print(query)

output = check_contains(query)

recs = get_recommendations_tfidf(query, data, top_n=10)
loop_print(recs, output)
recs = get_recommendations_tfidf(query, data, top_n=10, output_mode='int')
loop_print(recs, output)
recs = get_recommendations_tfidf(query, data, top_n=10, output_mode='count')
loop_print(recs, output)
recs = get_recommendations_tfidf(query, data, top_n=10, output_mode='multi_hot')
loop_print(recs, output)

Building
Conference, Building & Construction, IT & Technology
Conference, Building & Construction, Environmental Engineering
Tradeshow, Building & Construction, Furnishings & Decor, Bathroom & Kitchen, Flooring & Roofing
Tradeshow, Building & Construction, Home & Office
Conference, Building & Construction, Industrial Engineering
Conference, Building & Construction
Tradeshow, Building & Construction, Home & Office, Interior Design
Tradeshow, Building & Construction
Conference, Building & Construction, Civil Engineering

[(1, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (17, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (56, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (101, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (145, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (153, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (200, <tf.Tensor: shape=(), dtype=float32, numpy=11.48682>), (207, <tf.Tensor: shape=(), dtype=float32, numpy=11

In [46]:
def get_recommendations_tfidf(query, data, top_n=1, output_mode='tf_idf'):
    # Create the TextVectorization layer
    vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=None,  # Default value, no maximum limit on vocabulary size
        output_mode=output_mode,
        output_sequence_length=None,  # Default value, variable length sequences
        pad_to_max_tokens=False,  # Don't pad sequences
        standardize='lower_and_strip_punctuation',  # Disable standardization
        split='whitespace',  # Split text by whitespace
    )

    # Fit and transform the data
    new_data = data + [query]
    # print(new_data)
    vectorizer.adapt(new_data)
    X = vectorizer(new_data)

    # Compute the pairwise cosine similarity
    cosine_sim = tf.linalg.matmul(X, X, transpose_b=True)

    # Get the pairwise cosine similarity scores for the document
    sim_scores = list(enumerate(cosine_sim[-1, :-1]))

    # Sort the documents by similarity score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    top_docs = sim_scores[:top_n]
    print(top_docs)
    # Return the indices and recommendations
    output = {}
    c=0
    for i, j in top_docs:
        tags = new_data[i]
        match_score = float(j)

        tags_str = str(tags.split(', '))
        events_with_tags_df = events_df[events_df['tags']==tags_str]
        ids = list(events_with_tags_df['id'])
        print(ids)
        event_names = list(events_with_tags_df['event_name'])
        for index in range(len(events_with_tags_df)):
            if c == 10:
                break
            output[c] = {
                'event_id': ids[index],
                'event_name': event_names[index],
                'tags': tags,
                'match_score': match_score
            }
            c += 1
    return output
output = get_recommendations_tfidf("Tradeshow", data, top_n=10, output_mode='multi_hot')
output

[(0, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (1, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (3, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (4, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (5, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (6, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (7, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (10, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (16, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>), (17, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)]
[1]
[2, 208]
[4, 30, 116, 234]
[5, 180]
[6, 165, 238]
[7, 98, 187, 324]
[8, 247]
[11]
[]
[18]


{0: {'event_id': 1,
  'event_name': 'Tyre & Rubber Indonesia',
  'tags': 'Tradeshow, Auto & Automotive, Rubber & Tyres',
  'match_score': 1.0},
 1: {'event_id': 2,
  'event_name': 'Indonesia International Construction, Infrastructure & Mining Exhibition (CON-MINE)',
  'tags': 'Tradeshow, Building & Construction',
  'match_score': 1.0},
 2: {'event_id': 208,
  'event_name': 'International Flooring Technology (IFT)',
  'tags': 'Tradeshow, Building & Construction',
  'match_score': 1.0},
 3: {'event_id': 4,
  'event_name': 'World AI Show - Jakarta (WAIS)',
  'tags': 'Tradeshow, IT & Technology',
  'match_score': 1.0},
 4: {'event_id': 30,
  'event_name': 'Lazada Run Indonesia',
  'tags': 'Tradeshow, IT & Technology',
  'match_score': 1.0},
 5: {'event_id': 116,
  'event_name': 'Digital Transformation Indonesia Conference and Expo (DTI-CX)',
  'tags': 'Tradeshow, IT & Technology',
  'match_score': 1.0},
 6: {'event_id': 234,
  'event_name': 'Smart IoT Indonesia',
  'tags': 'Tradeshow, IT &

In [28]:
top_docs

NameError: name 'top_docs' is not defined

In [ ]:
matched_events = set()
matched_tags = set(j['event_name'] for i,j in output.items())
for tag in matched_tags:
    output_str = str(tag.split(', '))
    print(list(events_df[events_df['tags']==output_str]['event_name']))


TypeError: unhashable type: 'Series'